# SQL function creation

This notebook stores the code that creates the SQL functions that are then used by the API code. Needs to be run only once (and re-run if any updates of course!).

**Important**:In order to keep the list clean, please prefix any new functions with `api_`

In [1]:
import psycopg2
import configparser

config = configparser.ConfigParser()
config.read("../../config.ini")    
db_params = dict(config['DB'])

def execute_sql(SQL):
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                cur.execute(SQL)        

### Getting POIs for a given city

API endpoint: /pois

In [12]:
SQL = """

CREATE OR REPLACE FUNCTION api_get_pois_for_city(
    city_id int, poi_category character)
    RETURNS TABLE
            (
                id          int,
                h3id    char,
                name    varchar,
                lat         real,
                long        real,
                category    varchar
            )
    LANGUAGE plpgsql
AS
$poiforcity$
BEGIN    
    RETURN QUERY
        SELECT pois.poiid, pois.h3id, pois.name, pois.lat, pois.long, pois.category 
            FROM pois 
            JOIN cityh3map ON cityh3map.h3id = pois.h3id
		    JOIN cities ON cities.cityid = cityh3map.cityid
		    WHERE cities.cityID = city_id AND pois.category = poi_category;
END;
$poiforcity$;

"""

execute_sql(SQL)

### Demographics for a city

In [27]:
SQL = """
-- Demographics for a city

CREATE OR REPLACE FUNCTION api_get_demographics_for_city(
    in_cityid integer, in_categorytype character)
    RETURNS TABLE
            (
                h3id    char,
                groupname   text,
                population   float
            )
    LANGUAGE plpgsql
AS
$demographicsforcity$
BEGIN
    
    RETURN QUERY

        --- First, calculate # of H3 hexagons in each census block group in the city of interest

        WITH 
            
        censush3_counts AS
            (SELECT censush3map.censusblockgroupid, count(cityh3map.h3id) as hexagon_count
                FROM cityh3map
                JOIN cities ON cities.cityID = cityh3map.cityID
                JOIN censush3map ON censush3map.h3id = cityh3map.h3id
                WHERE cities.cityID = in_cityid
                GROUP BY censush3map.censusblockgroupid)

        --- Then, find associated population totals for the entire census block for each H3 hexagon in the city

        , demographic_totals AS
            (SELECT censush3map.censusblockgroupid, cityh3map.h3id, demographics.groupname, demographics.total
            FROM cityh3map
            JOIN cities ON cities.cityID = cityh3map.cityID
            JOIN censush3map ON censush3map.h3id = cityh3map.h3id
            JOIN demographics ON censush3map.censusblockgroupid = demographics.censusblockgroupid        
            WHERE demographics.categorytype = in_categorytype AND cities.cityID = in_cityid)

        
        --- Finally, join the above two tables together, and split the population equally among all hexagons
        
        SELECT demographic_totals.h3id, demographic_totals.groupname, demographic_totals.total::float / censush3_counts.hexagon_count::float as population
            FROM demographic_totals
            JOIN censush3_counts ON demographic_totals.censusblockgroupid = censush3_counts.censusblockgroupid;
END;
$demographicsforcity$;

-- SELECT * FROM get_demographics_for_city('Atlanta', 'Race'); 
"""

execute_sql(SQL)

### Getting catchment area statistics for a given H3ID origin

In [31]:
SQL = """
-- Demographics for a catchment area

CREATE OR REPLACE FUNCTION api_get_demographics_for_catchment(
    in_cityid integer, 
    in_categorytype character,
    in_origin_id character
)
    RETURNS TABLE
            (                
                groupname   text,
                population   float
            )
    LANGUAGE plpgsql
AS
$demographicsforarea$
BEGIN
    
    RETURN QUERY

        
        SELECT demo.groupname, SUM(demo.population) as population 
            FROM api_get_demographics_for_city(in_cityid, in_categorytype) as demo
            JOIN catchmenth3map ON catchmenth3map.h3id = demo.h3id
            JOIN catchments ON catchments.catchmentid = catchmenth3map.catchmentid
            WHERE catchments.originh3id = in_origin_id
        	GROUP BY demo.groupname;

END;
$demographicsforarea$;

-- SELECT * FROM api_get_demographics_for_catchment('Atlanta', 'Race', '8944c1a8c8fffff'); 
"""

execute_sql(SQL)